# Imports

In [2]:
import numpy as np
import pandas as pd
import ruptures as rpt
import plotly.express as px
import scipy.stats as stats

from pmdarima import auto_arima
from sklearn import linear_model
from pytrends.request import TrendReq
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

pytrend      = TrendReq(hl='en-worldwide', tz=360)
event_folder = '/Users/erichganz/Documents/Master Thesis/investment data'

comp_data_eval       = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/tolookup.xlsx', index_col=0)
timeseries_short     = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_short.xlsx', index_col=0)
timeseries_middle    = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_middle.xlsx', index_col=0)
timeseries_long      = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_long.xlsx', index_col=0)
timeseries_very_long = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_very_long.xlsx', index_col=0)

pd.options.mode.chained_assignment = None

# Building the Evaluation Matrix for GT Quality

## Start from the CB Insights Top 100

In [29]:
companies = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/Master_thesis_list.xlsx')
companies.tail()

,Company,Category,Product Focus Area,Total Disclosed\nFunding ($M),Country,Select Investors
195,InstaDeep,Cross-industry tech,Other R&D,NaN,United Kingdom,NaN
196,Kyndi,Cross-industry tech,Other R&D,NaN,United States,NaN
197,Nnaisense,Cross-industry tech,Other R&D,NaN,Switzerland,NaN
198,HACARUS,Cross-industry tech,Other R&D,NaN,Japan,NaN
199,SparkCognition,Cross-industry tech,Other R&D,NaN,United States,NaN


## Get Suggestions

In [30]:
sugestions = [pytrend.suggestions(name) for name in companies['Company']]

In [35]:
pytrend.suggestions('Screw')

[{'mid': '/m/026f7rv', 'title': 'Screwfix', 'type': 'Company'},
 {'mid': '/m/0c37z', 'title': 'Screw', 'type': 'Topic'},
 {'mid': '/m/01bms0', 'title': 'Screwdriver', 'type': 'Topic'},
 {'mid': '/m/08ms01', 'title': 'Prison officer', 'type': 'Topic'},
 {'mid': '/m/05xt4', 'title': 'Propeller', 'type': 'Marine propulsion'}]

In [39]:
terms =  ['/m/0c37z', '/m/052tr', '/m/01d30f']

In [36]:
pytrend.suggestions('Microsoft Excel')

[{'mid': '/m/052tr', 'title': 'Microsoft Excel', 'type': 'Computer program'},
 {'mid': '/g/11h6nps0ww', 'title': 'Microsoft Excel 2019', 'type': 'Topic'},
 {'mid': '/g/1ymz_jc06', 'title': 'Microsoft Excel 2013', 'type': 'Topic'},
 {'mid': '/m/02wx4fz', 'title': 'Microsoft Excel Viewer', 'type': 'Software'},
 {'mid': '/g/120r4t__', 'title': 'Xls', 'type': 'Format'}]

In [38]:
pytrend.suggestions('Teacher')

[{'mid': '/m/01d30f', 'title': 'Teacher', 'type': 'Profession'},
 {'mid': '/g/11fll501bv', 'title': 'TeachersPayTeachers', 'type': 'Topic'},
 {'mid': '/m/02y11r', 'title': "Teachers' Day", 'type': 'Celebration'},
 {'mid': '/m/0h8ds',
  'title': 'Edna Krabappel',
  'type': 'The Simpsons character'},
 {'mid': '/m/0h51tdt', 'title': 'CTET', 'type': 'Professional exam'}]

In [56]:
pytrend.build_payload(kw_list=[terms[1]], timeframe= '2017-01-01 2021-12-31')
test = pytrend.interest_over_time()

In [59]:
import plotly.express as px


cur = terms[1]

fig = px.line(test, x=test.index, y=test[f'{cur}'], title= 'Microsoft Excel', labels={'Microsoft Exce': 'Interest', 'date': 'Time since founding'})
fig.update_xaxes(rangeslider_visible=True)

fig.show()

nresult = seasonal_decompose(test[f'{cur}'], model='additive')

fig1 = px.line(nresult.seasonal, x=test.index, y=nresult.seasonal, title='Microsoft Excel - Sesonal Component', labels={'Microsoft Exce': 'Interest', 'date': 'Time since founding'})
    
fig1.show()

## Investigate Suggestions

In [33]:
comp_data_eval = pd.DataFrame(columns=['CompanyName', 'gtCode', 'GtName', 'GtType', 'TitleUnique'])

In [ ]:
company = 200
sugestions[company]

In [385]:
suggestion, value = 1, 1

dict_sug = sugestions[company][suggestion]

comp_data_eval.loc[len(comp_data_eval)] = [companies['Company'][company], dict_sug['mid'], dict_sug['title'], dict_sug['type'], value]

In [417]:
#if make mistake - delete
comp_data_eval.drop(14, axis=0, inplace=True)
comp_data_eval.reset_index(drop = True, inplace=True)

Checkpoint

In [2]:
comp_data_eval = pd.read_csv('/Users/erichganz/Documents/Master Thesis/Data/comp_codes_.csv', index_col=0)

## Get Related Queries

In [16]:
related_queries_grade = []
related_queries = []
gtcode_error = []

for code in comp_data_eval['gtCode']:

    try: 
        pytrend.build_payload(kw_list=[code])
        len_n = len((pytrend.related_queries()[code])['top'])

    except Exception as e: 
        gtcode_error.append([code, e])
        related_queries_grade.append(None)
        related_queries.append(None)

    else:
        related_queries_grade.append((1 if len_n > 10 else 0.5 if len_n > 4 else 0)) 
        related_queries.append(len_n)

comp_data_eval['RelatedQueries'] = related_queries
comp_data_eval['RelatedQueriesGrade'] = related_queries_grade

In [24]:
comp_data_eval = comp_data_eval.loc[[x not in gtcode_error for x in comp_data_eval.gtCode]]

In [26]:
comp_data_eval.to_csv('comp_codes_sug_relate.csv')

In [41]:
comp_data_eval = pd.read_csv('/Users/erichganz/Documents/Master Thesis/Data/comp_codes_sug_relate.csv', index_col=0)

In [42]:
comp_data_eval.reset_index(inplace=True, drop=True)

## Append Manually Foundation Dates

In [81]:
founding_dates = []

In [280]:
len(comp_data_eval['GtName'])

91

In [278]:
founding_dates.append(2015)
len(founding_dates)

91

In [281]:
comp_data_eval['FoundationDate'] = founding_dates

## Get Rid of too Old Companies

In [282]:
comp_data_eval = comp_data_eval.loc[comp_data_eval.FoundationDate >= 2012]

In [305]:
comp_data_eval

,CompanyName,gtCode,GtName,GtType,TitleUnique,RelatedQueries,RelatedQueriesGrade,FoundationDate
0,AiFi,/g/11g7y_93_d,AiFi,Company,1.0,15.0,1.0,2016
1,Algolux,/g/11b6dqhxy2,Algolux,"Software company in Montreal, Quebec",0.3,1.0,0.0,2014
2,Algolux,/g/11dxp_m583,Algolux Inc.,Company,1.0,1.0,0.0,2014
3,Algorithmia,/g/11c52db9bk,Algorithmia,Topic,0.3,25.0,1.0,2013
4,AMP Robotics,/g/11c6t_4333,AMP Robotics,"Industrial equipment supplier in Louisville, C...",0.3,4.0,0.0,2015
...,...,...,...,...,...,...,...,...
85,TuSimple,/g/11r6y6z60m,TuSimple Hldgs,Company,1.0,25.0,1.0,2015
86,TuSimple,/g/11c74fmpvj,TuSimple,"Software company in San Diego, California",0.3,6.0,0.5,2015
88,Kneron,/g/11c5r0m21k,KNERON,"Software company in San Diego, California",0.3,5.0,0.5,2015
89,DataRobot,/g/11f017ds55,"DataRobot, Inc.",Company,1.0,25.0,1.0,2012


## Get Related Time Serieses

In [158]:
def send_request(companies, time):

    if time == 'short': # Companies founded after 01.01.2018

        pytrend.build_payload(kw_list=companies, timeframe= '2017-01-01 2021-12-31')
        a = pytrend.interest_over_time()
        a = a.drop('isPartial', axis=1)

        return a
    
    elif time == 'middle': # Companies founded after 01.01.2018

        pytrend.build_payload(kw_list=companies, timeframe= '2017-01-01 2021-12-31')
        b = pytrend.interest_over_time()
        b = b.drop('isPartial', axis=1)

        pytrend.build_payload(kw_list=companies, timeframe='2014-01-01 2018-12-31')
        a = pytrend.interest_over_time()
        a = a.drop('isPartial', axis=1)
        
        scale = a['2017-01-01':].mean()/b[:'2018-12-31'].mean()

        b = scale * b

        unified_a_b = pd.concat([a, b['2018-12-31':]],axis=0)

        return unified_a_b
    
    elif time == 'long': # Companies founded after 01.01.2018

        pytrend.build_payload(kw_list=companies, timeframe='2017-01-01 2021-12-31')
        c = pytrend.interest_over_time()
        c = c.drop('isPartial', axis=1)

        pytrend.build_payload(kw_list=companies, timeframe='2014-01-01 2018-12-31')
        b = pytrend.interest_over_time()
        b = b.drop('isPartial', axis=1)

        pytrend.build_payload(kw_list=companies, timeframe='2011-01-01 2015-12-31')
        a = pytrend.interest_over_time()
        a = a.drop('isPartial', axis=1)

        scale = a['2014-01-01':].mean()/b[:'2015-12-31'].mean()

        b = scale * b

        unified_a_b = pd.concat([a,b['2015-12-31':]],axis=0)
        
        scale = unified_a_b['2017-01-01':].mean()/c[:'2018-12-31'].mean()

        c = scale * c

        unified_a_b_c = pd.concat([unified_a_b, c['2018-12-31':]],axis=0)
        
        return unified_a_b_c
    
    elif time == 'very_long': # Companies founded earlier than 01.01.2012

        pytrend.build_payload(kw_list=companies, timeframe='2017-01-01 2021-12-31')
        e = pytrend.interest_over_time()
        e = e.drop('isPartial', axis=1)

        pytrend.build_payload(kw_list=companies, timeframe='2014-01-01 2018-12-31')
        d = pytrend.interest_over_time()
        d = d.drop('isPartial', axis=1)

        pytrend.build_payload(kw_list=companies, timeframe='2011-01-01 2015-12-31')
        c = pytrend.interest_over_time()
        c = c.drop('isPartial', axis=1)

        pytrend.build_payload(kw_list=companies, timeframe='2008-01-01 2012-12-31')
        b = pytrend.interest_over_time()
        b = b.drop('isPartial', axis=1)

        pytrend.build_payload(kw_list=companies, timeframe='2005-01-01 2009-12-31')
        a = pytrend.interest_over_time()
        a = a.drop('isPartial', axis=1)

        #

        scale = a['2008-01-01':].mean()/b[:'2009-12-31'].mean()

        b = scale * b

        unified_a_b = pd.concat([a,b['2009-12-31':]],axis=0)

        #
        
        scale = unified_a_b['2011-01-01':].mean()/c[:'2012-12-31'].mean()

        c = scale * c

        unified_a_b_c = pd.concat([unified_a_b, c['2012-12-31':]],axis=0)

        #

        scale = unified_a_b_c['2014-01-01':].mean()/d[:'2015-12-31'].mean()

        d = scale * d

        unified_a_b_c_d = pd.concat([unified_a_b_c, d['2015-12-31':]],axis=0)

        #

        scale = unified_a_b_c_d['2017-01-01':].mean()/e[:'2018-12-31'].mean()

        e = scale * e

        unified_a_b_c_d_e = pd.concat([unified_a_b_c_d, e['2018-12-31':]],axis=0)

        #

        #unified_a_b_c_d_e = unified_a_b_c_d_e/unified_a_b_c_d_e.max()
        
        return unified_a_b_c_d_e

In [150]:
timeseries_short, timeseries_middle, timeseries_long, timeseries_very_long = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [176]:
load_now = comp_data_eval.loc[(comp_data_eval.FoundationYear < 2012) ]

In [178]:
for index, company in load_now.iterrows():

    if company.Code == '/g/11g7y_93_d': continue

    if company.FoundationYear >= 2018: # Companies founded after 01.01.2018

        short = [company.Code]
        gt_data = send_request(short, 'short')
        timeseries_short = pd.concat([timeseries_short, gt_data], axis = 1)
    
    elif company.FoundationYear >= 2015: # Companies founded after 01.01.2015
        
        middle = [company.Code]
        gt_data = send_request(middle, 'middle')
        timeseries_middle = pd.concat([timeseries_middle, gt_data], axis = 1)
    
    elif company.FoundationYear >= 2012: # Companies founded after 01.01.2012

        long = [company.Code]
        gt_data = send_request(long, 'long')
        timeseries_long = pd.concat([timeseries_long, gt_data], axis = 1)
    
    else: # Companies founded earlier than 01.01.2012

        very_long = [company.Code]
        gt_data = send_request(very_long, 'very_long')
        timeseries_very_long = pd.concat([timeseries_very_long, gt_data], axis = 1)

## Calculate the Means of the Time Serieses

In [342]:
comp_data_eval.reset_index(drop = True, inplace=True)

In [370]:
comp_data_eval['first_mean'] = pd.Series(dtype='float64')
comp_data_eval['over_mean']  = pd.Series(dtype='float64')

for times in [timeseries_short, timeseries_middle, timeseries_long]:

    for code in times.columns:

        foundation = comp_data_eval.loc[comp_data_eval.gtCode == code].iloc[0,7]
        time       = times[code][str(foundation-1) + '-01-01':]
        index      = comp_data_eval.loc[comp_data_eval.gtCode == code].index[0]

        comp_data_eval.iloc[index,8:10] = [time[:104].mean(), time.mean()] #104 weeks = 2 years

## Evaluate the Time Serieses

In [372]:
comp_data_eval['Mean_Ratio'] = comp_data_eval['first_mean']/comp_data_eval['over_mean']

In [373]:
comp_data_eval['Mean_Ratio_Grade'] = [1 if mean <= 0.5 else 0.5 if mean <= 0.85 else 0 for mean in comp_data_eval['Mean_Ratio']]

In [374]:
comp_data_eval['Mean_Grade'] = [1 if mean >= 0.04 else 0.5 if mean > 0.02 else 0 for mean in comp_data_eval['over_mean']]

In [375]:
comp_data_eval['Final_Grade'] = np.mean([np.mean([comp_data_eval['Mean_Grade'], comp_data_eval['TitleUnique'], comp_data_eval['RelatedQueriesGrade']], axis=0), comp_data_eval['Mean_Ratio_Grade']], axis=0)

## Select Companies with Good Grades

In [377]:
comp_data_eval = comp_data_eval.loc[comp_data_eval.Final_Grade >= 0.6]

In [356]:
comp_data_eval = pd.read_csv('/Users/erichganz/Documents/Master Thesis/Data/comps_finalgrade.csv', index_col=0)

In [376]:
comp_data_eval.to_excel('comps_finalgrade.xlsx')

In [397]:
for times in [timeseries_short, timeseries_middle, timeseries_long]:
    for code in times.columns:

        if code not in list(comp_data_eval.gtCode):

            times.drop(code, axis=1, inplace=True)

# Save Point

In [196]:
#Safe the current data
timeseries_short.to_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_short.xlsx')
timeseries_middle.to_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_middle.xlsx')
timeseries_long.to_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_long.xlsx')
timeseries_very_long.to_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_very_long.xlsx')

In [3]:
#Load the current data
timeseries_short     = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_short.xlsx', index_col=0)
timeseries_middle    = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_middle.xlsx', index_col=0)
timeseries_long      = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_long.xlsx', index_col=0)
timeseries_very_long = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_very_long.xlsx', index_col=0)

# Analysis of Timeseries

## Useful functions

In [4]:
def code_to_time(code, extension = ''):

    foundation = int(comp_data_eval.loc[comp_data_eval.Code == code].FoundationYear)
    if foundation   >= 2018: return timeseries_short[[f'{code}{extension}']][str(foundation) + '-01-01':]
    elif foundation >= 2015: return timeseries_middle[[f'{code}{extension}']][str(foundation) + '-01-01':]
    elif foundation >= 2012: return timeseries_long[[f'{code}{extension}']][str(foundation) + '-01-01':]
    else:                    return timeseries_very_long[[f'{code}{extension}']][str(foundation) + '-01-01':]

def code_to_full_time(code, extension = ''):

    foundation = int(comp_data_eval.loc[comp_data_eval.Code == code].FoundationYear)
    if foundation   >= 2018: return timeseries_short[[f'{code}{extension}']]
    elif foundation >= 2015: return timeseries_middle[[f'{code}{extension}']]
    elif foundation >= 2012: return timeseries_long[[f'{code}{extension}']]
    else:                    return timeseries_very_long[[f'{code}{extension}']]

def code_to_time_two(code, foundation, extension = ''):

    foundation = int(foundation)

    if foundation   >= 2018: 
        return timeseries_short[[f'{code}{extension}']][:str(foundation -1) + '-12-31'], timeseries_short[[f'{code}{extension}']][str(foundation) + '-01-01':]
    elif foundation >= 2015: 
        return timeseries_middle[[f'{code}{extension}']][:str(foundation -1) + '-12-31'], timeseries_middle[[f'{code}{extension}']][str(foundation) + '-01-01':]
    elif foundation >= 2012: 
        return timeseries_long[[f'{code}{extension}']][:str(foundation-1) + '-12-31'], timeseries_long[[f'{code}{extension}']][str(foundation) + '-01-01':]
    else:                    
        return timeseries_very_long[[f'{code}{extension}']][:str(foundation-1) + '-12-31'], timeseries_very_long[[f'{code}{extension}']][str(foundation) + '-01-01':]

## Selecting meaningfull time series 

In [11]:
good_time = [0, 3, 4, 7, 10, 13, 14, 15, 17, 18, 19, 20, 21, 26, 28, 29, 31, 33, 34, 38, 39, 40, 43, 46, 49, 52, 53, 54, 55, 56, 57, 58, 64, 73, 75, 76, 77]
good_time = comp_data_eval.loc[[index in good_time for index in comp_data_eval.index.values]]

# Event Data Analysis on Time Series

## Excluding Event Highs and End-of-Year Lows

In [6]:
def highest_neighbor(data, date):
    
    index  = int(data.index.get_indexer([date], method='pad'))

    first  = (index-1, data[index-1])
    second = (index, data[index])
    third  = (index+1, data[index+1])

    return max([first, second, third], key=lambda x:x[1])[0]

def lowest_neighbor(data, date):
    
    index  = int(data.index.get_indexer([date], method='nearest'))

    first  = (index-1, data[index-1])
    second = (index, data[index])
    third  = (index+1, data[index+1])

    return min([first, second, third], key=lambda x:x[1])[0]

def get_mean(data, index): return  (data[index-1]+ data[index+1])/2

def get_mean2(data, index): return  (data[index-2]+ data[index+2])/2



In [25]:
for index_comp, comp in comp_data_eval.iterrows():
    
    data   = code_to_full_time(comp.Code)[f'{comp.Code}']
    events = pd.read_excel(f'{event_folder}/{comp.Name}.xlsx')
    years  = list(range(int(comp.FoundationYear), 2021))

    '''
    # Exclude Events
    for index_event, event in events.iterrows():

        replace = highest_neighbor(data, event.Date)
        
        # replace the highest
        data.iloc[replace]   = get_mean2(data, replace)
        data.iloc[replace+1] = get_mean(data, replace+1)
        data.iloc[replace-1] = get_mean(data, replace-1)
    '''

    # Exclude Sesonality
    for year in years:

        replace = lowest_neighbor(data, f'{year}-12-31')
        
        # replace the highest
        data.iloc[replace]   = get_mean2(data, replace)
        data.iloc[replace+1] = get_mean(data, replace+1)
        data.iloc[replace-1] = get_mean(data, replace-1)

    if  comp.FoundationYear  >= 2018: timeseries_short[f'{comp.Code}_no_seasonality']     = data
    elif comp.FoundationYear >= 2015: timeseries_middle[f'{comp.Code}_no_seasonality']    = data
    elif comp.FoundationYear >= 2012: timeseries_long[f'{comp.Code}_no_seasonality']      = data
    else:                             timeseries_very_long[f'{comp.Code}_no_seasonality'] = data

## Detrend

In [12]:

for index, comp in comp_data_eval.iterrows():


    if comp.FoundationYear >= 2018:

        empty, data_no_event = code_to_time_two(comp.Code, comp.FoundationYear, '_no_events')
        data_no_event        = data_no_event[f'{comp.Code}_no_events']
        empty, data_raw      = code_to_time_two(comp.Code, comp.FoundationYear)
        empty, data_raw      = empty[f'{comp.Code}'], data_raw[f'{comp.Code}']

        result_add = seasonal_decompose(data_no_event, model='additive', extrapolate_trend='freq')
        
        timeseries_short[f'{comp.Code}_detrended'] = pd.concat([empty, data_raw - result_add.trend], axis=0)
        timeseries_short[f'{comp.Code}_no_events_detrended'] = pd.concat([empty, data_no_event - result_add.trend], axis=0)

    elif comp.FoundationYear >= 2015: 

        empty, data_no_event = code_to_time_two(comp.Code, comp.FoundationYear, '_no_events')
        data_no_event        = data_no_event[f'{comp.Code}_no_events']
        empty, data_raw      = code_to_time_two(comp.Code, comp.FoundationYear)
        empty, data_raw      = empty[f'{comp.Code}'], data_raw[f'{comp.Code}']

        result_add = seasonal_decompose(data_no_event, model='additive', extrapolate_trend='freq')
        
        timeseries_middle[f'{comp.Code}_detrended'] = pd.concat([empty, data_raw - result_add.trend], axis=0)
        timeseries_middle[f'{comp.Code}_no_events_detrended'] = pd.concat([empty, data_no_event - result_add.trend], axis=0)
    
    elif comp.FoundationYear >= 2012: 

        empty, data_no_event = code_to_time_two(comp.Code, comp.FoundationYear, '_no_events')
        data_no_event        = data_no_event[f'{comp.Code}_no_events']
        empty, data_raw      = code_to_time_two(comp.Code, comp.FoundationYear)
        empty, data_raw      = empty[f'{comp.Code}'], data_raw[f'{comp.Code}']

        result_add = seasonal_decompose(data_no_event, model='additive', extrapolate_trend='freq')
        
        timeseries_long[f'{comp.Code}_detrended'] = pd.concat([empty, data_raw - result_add.trend], axis=0)
        timeseries_long[f'{comp.Code}_no_events_detrended'] = pd.concat([empty, data_no_event - result_add.trend], axis=0)
    
    else: 
        
        empty, data_no_event = code_to_time_two(comp.Code, comp.FoundationYear, '_no_events')
        data_no_event        = data_no_event[f'{comp.Code}_no_events']
        empty, data_raw      = code_to_time_two(comp.Code, comp.FoundationYear)
        empty, data_raw      = empty[f'{comp.Code}'], data_raw[f'{comp.Code}']
        
        result_add = seasonal_decompose(data_no_event, model='additive', extrapolate_trend='freq')
        
        timeseries_very_long[f'{comp.Code}_detrended'] = pd.concat([empty, data_raw - result_add.trend], axis=0)
        timeseries_very_long[f'{comp.Code}_no_events_detrended'] = pd.concat([empty, data_no_event - result_add.trend], axis=0)
   

## ARIMA residuals

In [8]:
arima_orders = []

for index, comp in comp_data_eval.iterrows():

    if comp.FoundationYear >= 2018:

        empty, data_no_event = code_to_time_two(comp.Code, comp.FoundationYear, '_no_events')

        order = auto_arima(data_no_event).order
        arima_orders.append(order)
        data_no_event.index = pd.DatetimeIndex(data_no_event.index.values, freq='W')
        model = ARIMA(data_no_event, order = order)
        model = model.fit()

        timeseries_short[f'{comp.Code}_resid_ARIMA'] = list(empty[f'{comp.Code}_no_events']) + list(model.resid)

        empty, data = code_to_time_two(comp.Code, comp.FoundationYear, '_no_seasonality')
        timeseries_short[f'{comp.Code}_resid_ARIMA_events'] = list(empty[f'{comp.Code}_no_seasonality']) + list(data[f'{comp.Code}_no_seasonality'] - model.predict())


    elif comp.FoundationYear >= 2015: 

        empty, data_no_event = code_to_time_two(comp.Code, comp.FoundationYear, '_no_events')

        order = auto_arima(data_no_event).order
        arima_orders.append(order)
        data_no_event.index = pd.DatetimeIndex(data_no_event.index.values, freq='W')
        model = ARIMA(data_no_event, order = order)
        model = model.fit()

        timeseries_middle[f'{comp.Code}_resid_ARIMA'] = list(empty[f'{comp.Code}_no_events']) + list(model.resid)

        empty, data = code_to_time_two(comp.Code, comp.FoundationYear, '_no_seasonality')
        timeseries_middle[f'{comp.Code}_resid_ARIMA_events'] = list(empty[f'{comp.Code}_no_seasonality']) + list(data[f'{comp.Code}_no_seasonality'] - model.predict())
        
    elif comp.FoundationYear >= 2012: 

        empty, data_no_event = code_to_time_two(comp.Code, comp.FoundationYear, '_no_events')

        order = auto_arima(data_no_event).order
        arima_orders.append(order)
        data_no_event.index = pd.DatetimeIndex(data_no_event.index.values, freq='W')
        model = ARIMA(data_no_event, order = order)
        model = model.fit()

        timeseries_long[f'{comp.Code}_resid_ARIMA'] =  list(empty[f'{comp.Code}_no_events']) + list(model.resid)

        empty, data = code_to_time_two(comp.Code, comp.FoundationYear, '_no_seasonality')
        timeseries_long[f'{comp.Code}_resid_ARIMA_events'] = list(empty[f'{comp.Code}_no_seasonality']) + list(data[f'{comp.Code}_no_seasonality'] - model.predict())
    
    else: 

        empty, data_no_event = code_to_time_two(comp.Code, comp.FoundationYear, '_no_events')

        order = auto_arima(data_no_event).order
        arima_orders.append(order)
        data_no_event.index = pd.DatetimeIndex(data_no_event.index.values, freq='W')
        model = ARIMA(data_no_event, order = order)
        model = model.fit()
        
        timeseries_very_long[f'{comp.Code}_resid_ARIMA'] =  list(empty[f'{comp.Code}_no_events']) + list(model.resid)

        empty, data = code_to_time_two(comp.Code, comp.FoundationYear, '_no_seasonality')
        timeseries_very_long[f'{comp.Code}_resid_ARIMA_events'] = list(empty[f'{comp.Code}_no_seasonality']) + list(data[f'{comp.Code}_no_seasonality'] - model.predict())
    
#comp_data_eval['Best_ARIMA_order'] = arima_orders

/Users/erichganz/Library/Python/3.8/lib/python/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/erichganz/Library/Python/3.8/lib/python/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/erichganz/Library/Python/3.8/lib/python/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/erichganz/Library/Python/3.8/lib/python/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimizat

# Save Point

In [25]:
#Safe the current data
comp_data_eval.to_excel('/Users/erichganz/Documents/Master Thesis/Data/tolookup.xlsx')

timeseries_short.to_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_short.xlsx')
timeseries_middle.to_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_middle.xlsx')
timeseries_long.to_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_long.xlsx')
timeseries_very_long.to_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_very_long.xlsx')

In [5]:
#Safe the current data
comp_data_eval    = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/tolookup.xlsx')
timeseries_short  = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_short.xlsx', index_col=0)
timeseries_middle = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_middle.xlsx', index_col=0)
timeseries_long   = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_long.xlsx', index_col=0)
timeseries_very_long = pd.read_excel('/Users/erichganz/Documents/Master Thesis/Data/timeseries_very_long.xlsx', index_col=0)

# Calulate the Magnitute of CV Events

I assumed that the groud interest follows some normally distributed data generating process. An event can be considered as shock to the data generating process. This section tries to calculate the probabilistic magnitued of this show with respect to the forgone normally distributed data generating process. 

Nine points before the event are taken, on which the mean is calculated as well as the standart deviation. With the metrices the Z-values of the event point is calculated. The Z score is used a mesure of extreamity with respect to the distribution of the nine point before the event.

In [7]:

for index, comp in comp_data_eval.iterrows():

    events  = pd.read_excel(f'{event_folder}/{comp.Name}.xlsx')
    data_detre  = code_to_time(comp.Code, '_detrended')[f'{comp.Code}_detrended']
    data_detre_noeve = code_to_time(comp.Code, '_no_events_detrended')[f'{comp.Code}_no_events_detrended']

    scores, values, dates = [], [], []

    for row in events.iterrows():

        position    = highest_neighbor(data_detre, row[1].Date)
        event_value = data_detre[position]

        window = list(data_detre_noeve[position-12:position]) + [event_value]
        
        scores.append(stats.zscore(window)[-1])
        values.append(event_value)
        dates.append(data_detre.index.values[position])
    
    events['Magnitute'], events['pickedValues'], events['picked_date'], events['comp_name'] = scores, values, dates, [comp.Name] * len(scores)

    events.to_excel(f'{event_folder}/{comp.Name}.xlsx', index=False)


In [9]:

for index, comp in comp_data_eval.iterrows():

    events       = pd.read_excel(f'{event_folder}/{comp.Name}.xlsx')
    errors       = code_to_full_time(comp.Code, '_resid_ARIMA_events')[f'{comp.Code}_resid_ARIMA_events']
    errors_clean = code_to_full_time(comp.Code, '_resid_ARIMA')[f'{comp.Code}_resid_ARIMA']
    scores       = []

    for row in events.iterrows():

        position    = highest_neighbor(errors, row[1].Date)
        event_value = errors[position]

        window = list(errors_clean[position-37:position]) + [event_value]
        
        scores.append(stats.zscore(window)[-1])

        if scores[-1] != scores[-1]: scores[-1] = 0
    
    events['Magnitute_Arima'] = scores

    events.to_excel(f'{event_folder}/{comp.Name}.xlsx', index=False)

In [11]:
len(errors_clean[position-37:position])

37

# Aggregate events

In [221]:
all_events = pd.DataFrame()

for name in comp_data_eval.Name.values:

    events = pd.read_excel(f'{event_folder}/{name}.xlsx')

    all_events = pd.concat([all_events, events], axis=0)

all_events.set_index(all_events.Date, inplace = True)
all_events.drop(['Date'], inplace = True, axis=1)

Agregate zscore

In [222]:
mean_zscore, max_zscore = [], []

for index, comp in comp_data_eval.iterrows():

    mean_zscore.append(all_events.loc[all_events.comp_name == comp.Name].Magnitute.mean())
    max_zscore.append(all_events.loc[all_events.comp_name == comp.Name].Magnitute.max())

comp_data_eval['mean_zscore'] = mean_zscore
comp_data_eval['max_zscore']  = max_zscore



In [224]:
comp_data_eval.mean_zscore.mean()

1.679542977106086

In [225]:
comp_data_eval.max_zscore.mean()

2.973867738564835

### Events

### Fundings

In [695]:
all_events.loc[list(all_events.Event.isna()) and ['Series A' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Series A'
all_events.loc[list(all_events.Event.isna()) and ['Series B' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Series B'
all_events.loc[list(all_events.Event.isna()) and ['Series C' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Series C'
all_events.loc[list(all_events.Event.isna()) and ['Series D' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Series D'
all_events.loc[list(all_events.Event.isna()) and ['Series E' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Series E'
all_events.loc[list(all_events.Event.isna()) and ['Series F' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Series F'
all_events.loc[list(all_events.Event.isna()) and ['Series G' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Series G'

all_events.loc[list(all_events.Event.isna()) and ['Corporate Minority' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Corporate Minority'
all_events.loc[list(all_events.Event.isna()) and ['Incubator/Accelerator' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Incubator/Accelerator'
all_events.loc[list(all_events.Event.isna()) and ['Seed VC' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Seed VC'
all_events.loc[list(all_events.Event.isna()) and ['Seed' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Seed'
all_events.loc[list(all_events.Event.isna()) and ['Grant' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Grant'
all_events.loc[list(all_events.Event.isna()) and ['Unattributed' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Unattributed'
all_events.loc[list(all_events.Event.isna()) and ['Angel' in fund if fund == fund else False for fund in all_events.Funding], 'Funding'] = 'Angel'

In [696]:
all_events.loc[all_events.Event.isna()].Funding.value_counts().head(10).index

Index(['Series A', 'Seed', 'Series B', 'Series C', 'Incubator/Accelerator',
       'Series D', 'Unattributed', 'Corporate Minority', 'Series E',
       'Series F'],
      dtype='object')

In [716]:
for fund_round in all_events.loc[all_events.Event.isna()].Funding.value_counts().head(10).index:

    if fund_round in ['Series A', 'Series B', 'Series C', 'Series D', 'Series E', 'Series F']:

        all_events[fund_round] = [1 if fund_round == round else 0 for round in all_events.Funding.values]

#all_events.drop('Funding', axis=1, inplace = True)

### Investors

In [717]:
# Get list of all investors with dublicates
investors, num_of_investors = [], []

for value in all_events.Investors.values:

    if value == value:
        
        all_investors = value.split(',')
        investors.extend(all_investors)
        num_of_investors.append(len(all_investors))
    
    else: num_of_investors.append(-1)

all_events['num_of_investors'] = num_of_investors

investos = pd.Series(investors)

# Create dummies for the to 20 investors
top_investors = investos.value_counts().head(10)

for topInvestor in top_investors.index:

    if topInvestor == 'Undisclosed Investors': continue

    dummy = []

    for investors in all_events.Investors.values:

        if investors == investors: dummy.append(1 if topInvestor in investors else 0)
        else: dummy.append(0)
    
    all_events[topInvestor] = dummy

all_events.drop('Investors', axis=1, inplace = True)

### Valuation

In [719]:
# replace first valuations with 0 if the are NA
all_events.loc[(all_events.Event.isna()) & all_events.Valuation.isna() & (all_events['comp_name'].shift(-1, fill_value=all_events["comp_name"].head(1)) != all_events["comp_name"]), 'Valuation'] = 0

# Interpolate valuation points
for name in good_time.CompanyName.values:

    all_events.loc[ (all_events.Event.isna()) & (all_events.comp_name == name), 'Valuation' ] = all_events.loc[ (all_events.Event.isna()) & (all_events.comp_name == name) ].Valuation.interpolate(method='time', imit_direction='forward')

# Predict latest missing values
for name in good_time.CompanyName.values:

    y    = all_events.loc[ (all_events.Event.isna()) & (all_events.comp_name == name) & (~all_events.Valuation.isna()), 'Valuation']
    pred = all_events.loc[ (all_events.Event.isna()) & (all_events.comp_name == name) & (all_events.Valuation.isna()), 'Valuation']

    if len(pred) == 0 or len(y) in [0,1]: continue

    x = (y.index - y.index[-1]).days
    x = x.values.reshape(-1, 1)

    pred = (pred.index - y.index[-1]).days
    pred = pred.values.reshape(-1, 1)

    reg = linear_model.LinearRegression().fit(x, y)

    all_events.loc[ (all_events.Event.isna()) & (all_events.comp_name == name) & (all_events.Valuation.isna()), 'Valuation'] = reg.predict(pred)

In [720]:
# Missing vluations for companies
all_events.loc[ (all_events.Event.isna()) & (all_events.Valuation.isna())]

,Funding,Event,Amount,Valuation,Sources,Link,Magnitute,pickedValues,picked_date,comp_name,...,num_of_investors,Google Ventures,Tiger Global Management,Plug and Play,New Enterprise Associates,Sequoia Capital,FirstMark Capital,Accel,Samsung,BOLDstart Ventures
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-19,Series A,NaN,15.00,NaN,5.0,NaN,1.668885,0.327904,2020-10-18,AiFi,...,4,0,0,0,0,0,0,0,0,0
2020-05-03,Loan,NaN,0.68,NaN,NaN,NaN,0.564803,0.293519,2020-05-03,AiFi,...,1,0,0,0,0,0,0,0,0,0
2019-09-26,Incubator/Accelerator - II,NaN,NaN,NaN,1.0,NaN,0.227125,0.240100,2019-09-15,AiFi,...,1,0,0,1,0,0,0,0,0,0
2019-06-11,Seed VC - II,NaN,11.00,NaN,1.0,NaN,1.949178,0.243964,2019-06-16,AiFi,...,2,0,0,0,0,0,0,0,0,0
2019-03-25,Biz Plan Competition,NaN,NaN,NaN,1.0,NaN,1.210865,0.507605,2019-03-17,AiFi,...,1,0,0,0,0,0,0,0,0,0
2019-03-05,Incubator/Accelerator,NaN,NaN,NaN,1.0,NaN,0.874089,0.453532,2019-03-03,AiFi,...,1,0,0,1,0,0,0,0,0,0
2021-11-08,Series A,NaN,57.00,NaN,4.0,NaN,2.908077,1.065916,2021-11-07,Landing AI,...,2,0,0,0,0,0,0,0,0,0
2019-04-01,Seed VC - II,NaN,NaN,NaN,2.0,NaN,0.576101,0.227739,2019-03-31,Landing AI,...,1,0,0,0,0,0,0,0,0,0
2018-01-30,Seed VC,NaN,NaN,NaN,1.0,NaN,1.760193,0.517104,2018-01-28,Landing AI,...,1,0,0,0,0,0,0,0,0,0


#### Visualize the valuations over time

In [535]:
for name in all_events.comp_name.unique():

    dataset = all_events.loc[(all_events.comp_name == name) & (~ all_events.Valuation.isna())]

    fig = px.line(dataset, x=dataset.index, y="Valuation", title=name)
    fig.show()

### Amount

In [721]:
all_events.loc[all_events.Event.isna()].Amount.isna().sum()

46

In [722]:
# Mssing amounts are set to zero because the fact that the data is missing indicated that the amount was not disclosed.
# Thefore any attention this investment anouncment might have recived is unrelated the amount which has been related. 
all_events.loc[(all_events.Event.isna()) & (all_events.Amount.isna()), 'Amount'] = 0 

### Sources

In [723]:
# If no sources are mentioned, then it must be zero 
all_events.loc[all_events.Event.isna() & all_events.Sources.isna() ] = 0

# More

In [ ]:
for index_good_time, row in good_comps.iterrows():

    # load results
    events = pd.read_excel(f'{reg_folder}/{row.Name}.xlsx')

    amounts    = []
    valuations = []

    for index, event in events.iterrows():

        amount = ''

        if event.Amount == event.Amount:

            for sign in str(event.Amount):

                if sign.isdigit() or sign == '.':

                    amount += sign 
        
        else:

            amount = np.nan
        
        amounts.append(float(amount))

        valuation = ''

        if event.Valuation == event.Valuation:

            for sign in str(event.Valuation):

                if sign.isdigit() or sign == '.':

                    valuation += sign 
        
        else:

            valuation = np.nan
        
        valuations.append(float(valuation))

    
    events['Amount'] = amounts
    events['Valuation'] = valuations


    # Save results
    events.to_excel(f'{reg_folder}/{row.Name}.xlsx')